# 预处理 Preprocessing

现实生活中，我们会通过多种渠道来获取数据，例如传统的调查问卷、网络爬虫系统或者关系数据库等。
在对这些数据进行分析之前通常需要进行**数据预处理**。
例如，调查问卷中的某些调查对象可能会选择不回答特定的问题，这就造成数据存在缺失的情况；
从网页直接下载的数据同时包含网页结构和网页内容，这就需要对半结构化的数据进行结构化处理等等。
此外，数据在采集和传输过程中可能会引入噪音，因此需要离群值检测方法将噪音识别出来；
且在使用一些需要进行距离计算的模型之前，要对数据进行标准化处理。

<kbd><font color=Red>sklearn.preprocessing</font></kbd>包提供了几个常见的实用功能和变换器类型，用来将原始特征向量更改为更适合机器学习模型的形式。详情参考[英文文档](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing)或[中文文档](https://www.sklearncn.cn/40/#531)。

## 1 标准化 Standardization

数据集的**标准化**对scikit-learn中实现的大多数机器学习算法来说是常见的要求。
如果个别特征看起来不是很像标准正态分布（具有零均值和单位方差），那么它们的表现力可能会较差。

在实际情况中，我们经常忽略特征的分布形状，直接经过去均值来对某个特征进行中心化，再通过除以非常量特征（non-constant features）的标准差进行缩放。

函数<kbd><font color=Blue>scale</font></kbd>为数组形状的数据集的标准化提供了一个快捷实现：

In [2]:
from sklearn import preprocessing
import numpy as np
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])
X_scaled = preprocessing.scale(X_train)

In [3]:
X_scaled

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

经过缩放后的数据具有零均值以及标准方差：

In [4]:
# 均值
X_scaled.mean(axis=0)

array([0., 0., 0.])

In [5]:
# 方差
X_scaled.std(axis=0)

array([1., 1., 1.])

一种标准化是将特征缩放到给定的最小值和最大值之间，通常在0和1之间，也可以将每个特征的最大绝对值转换至单位大小。
可以分别使用<kbd><font color=Blue>MinMaxScaler</font></kbd>和<kbd><font color=Blue>MaxAbsScaler</font></kbd>实现。

In [6]:
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_train_minmax

array([[0.5       , 0.        , 1.        ],
       [1.        , 0.5       , 0.33333333],
       [0.        , 1.        , 0.        ]])

如果给<kbd><font color=Blue>MinMaxScaler</font></kbd>提供一个明确的<kbd><font color=Red>feature_range=(min, max)</font></kbd>，完整的公式是：

In [7]:
X = X_train
max, min = 10, 1
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_scaled = X_std * (max - min) + min

<kbd><font color=Blue>MaxAbsScaler</font></kbd>的工作原理非常相似，但是它只通过除以每个特征的最大值将训练数据特征缩放至<kbd><font color=Red>[-1,1]</font></kbd>范围内，这就意味着，训练数据应该是已经零中心化或者是稀疏数据。

以下是使用上例中数据运用这个缩放器的示例：

In [9]:
max_abs_scaler = preprocessing.MaxAbsScaler()
X_train_maxabs = max_abs_scaler.fit_transform(X_train)
X_train_maxabs

array([[ 0.5, -1. ,  1. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  1. , -0.5]])

## 2 归一化 Normalization

**归一化**是**缩放单个样本以具有单位范数**的过程。如果你计划使用二次形式（如点积或任何其他核函数）来量化任何样本间的相似度，则此过程非常有用。

函数<kbd><font color=Blue>normalize</font></kbd>提供了一个快速简单的方法在类似数组的数据集上执行操作，使用<kbd><font color=Red>l1</font></kbd>或<kbd><font color=Red>l2</font></kbd>范数：

In [10]:
X_normalized = preprocessing.normalize(X_train, norm='l2')
X_normalized

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

## 3 类别特征编码 Encoding categorical features

在机器学习中，特征可能不是连续的数值型的而是categorical类型，如性别<kbd><font color=Red>["male","female"]</font></kbd>，我们可以将其编码成整数<kbd><font color=Red>[0,1]</font></kbd>。

要把categorical型特征转换为这样的整数编码(integer codes), 我们可以使用<kbd><font color=Blue>OrdinalEncoder</font></kbd>。
这个估计器把每一个categorical feature变换成一个新的整数数字特征（0到 n_categories - 1）:

In [11]:
enc = preprocessing.OrdinalEncoder()
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)
enc.transform([['female', 'from US', 'uses Safari']])

array([[0., 1., 1.]])

在上面这个例子中，第一个特征是性别，female被编码为0；第二个特征是来源地，US被编码为1；第三个特征使用的浏览器，Safari被编码为1。

这样的整数特征表示并不能在scikit-learn的估计器中直接使用，因为这样的连续输入，估计器会认为类别之间是有序的，但实际却是无序的。
(例如：浏览器的类别数据是任意排序的)。

另外一种将标称型特征转换为能够被scikit-learn中模型使用的编码是**one-of-K**，又称为**独热码**或**dummy encoding**。
这种编码类型已经在类<kbd><font color=Blue>OneHotEncoder</font></kbd>中实现。
该类把每一个具有n_categories个可能取值的categorical特征变换为长度为n_categories的二进制特征向量，里面只有一个地方是1，其余位置都是0。

In [12]:
enc = preprocessing.OneHotEncoder()
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)
enc.transform([['female', 'from US', 'uses Safari'],
               ['male', 'from Europe', 'uses Safari']]).toarray()

array([[1., 0., 0., 1., 0., 1.],
       [0., 1., 1., 0., 0., 1.]])

现在我们有了六个特征，第一个位置为1即为female，第二个位置为1即为male，其他位置类似。

默认情况下，每个特征使用几维的数值可以从数据集自动推断。而且也可以在属性categories_中找到：

In [13]:
enc.categories_

[array(['female', 'male'], dtype=object),
 array(['from Europe', 'from US'], dtype=object),
 array(['uses Firefox', 'uses Safari'], dtype=object)]

可以使用参数<kbd><font color=Red>categories_</font></kbd>显式地指定这一点。
我们的数据集中有两种性别、四种可能的大陆和四种web浏览器:

In [14]:
genders = ['female', 'male']
locations = ['from Africa', 'from Asia', 'from Europe', 'from US']
browsers = ['uses Chrome', 'uses Firefox', 'uses IE', 'uses Safari']
enc = preprocessing.OneHotEncoder(categories=[genders, locations, browsers])

# Note that for there are missing categorical values for the 2nd and 3rd
# feature
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)
enc.transform([['female', 'from Asia', 'uses Chrome']]).toarray()

array([[1., 0., 0., 1., 0., 0., 1., 0., 0., 0.]])

如果训练数据可能缺少分类特性，通常最好指定<kbd><font color=Red>handle_unknown</font></kbd>='ignore'，而不是像上面那样手动设置类别。
当指定handle_unknown='ignore'，并且在转换过程中遇到未知类别时，不会产生错误，但是为该特性生成的一热编码列将全部为零(handle_unknown='ignore'只支持one hot编码)：

In [15]:
enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)
enc.transform([['female', 'from Asia', 'uses Chrome']]).toarray()

array([[1., 0., 0., 0., 0., 0.]])

## 4 离散化 Discretization

**离散化**（有些时候叫量化（quantization）或装箱（binning））提供了将连续特征划分为离散特征值的方法。
某些具有连续特征的数据集会受益于离散化，因为离散化可以把具有连续属性的数据集变换成只有名义属性(nominal attributes)的数据集。
(注：nominal attributes 其实就是 categorical features)。

One-hot编码的离散化特征可以使得一个模型更加的有表现力(expressive)，同时还能保留其可解释性(interpretability)。
比如，用离散化器进行预处理可以给线性模型引入非线性。

<kbd><font color=Blue>KBinsDiscretizer</font></kbd>类使用k个等宽的bins把特征离散化：

In [16]:
X = np.array([[ -3., 5., 15 ],
              [  0., 6., 14 ],
              [  6., 3., 11 ]])
est = preprocessing.KBinsDiscretizer(n_bins=[3, 2, 2], encode='ordinal').fit(X)

默认情况下，输出是被 one-hot 编码到一个稀疏矩阵，而且可以使用参数<kbd><font color=Red>encode</font></kbd>进行配置。
对每一个特征，<kbd><font color=Red>bin</font></kbd>的边界以及总数目在<kbd><font color=Red>fit</font></kbd>过程中被计算出来，它们将用来定义区间。
因此，对现在的示例，这些区间间隔被定义如下：
特征1<kbd><font color=Red>[-∞,-1],[-1,2),[2,∞)</font></kbd>，
特征2<kbd><font color=Red>[-∞,5),[5,∞)</font></kbd>，
特征3<kbd><font color=Red>[-∞,14],[14,∞)</font></kbd>。

基于这些<kbd><font color=Red>bin</font></kbd>区间, X 就被变换成下面这样：

In [17]:
est.transform(X)

array([[0., 1., 1.],
       [1., 1., 1.],
       [2., 0., 0.]])

由此产生的数据集包含了有序属性(ordinal attributes),可以被进一步用在类<kbd><font color=Blue>sklearn.pipeline.Pipeline</font></kbd>中。

离散化(Discretization)类似于为连续数据构建直方图(histograms)。
然而，直方图聚焦于统计特征落在特定的bins里面的数量，而离散化聚焦于给这些bins分配特征取值。

<kbd><font color=Blue>KBinsDiscretizer</font></kbd>类实现了不同的binning策略，可以通过参数<kbd><font color=Red>strategy</font></kbd>进行选择。
‘uniform’策略使用固定宽度的bins。
‘quantile’策略在每个特征上使用分位数(quantiles)值以便具有相同填充的bins。
‘kmeans’策略基于在每个特征上独立执行的k-means聚类过程定义bins。

## 5 缺失值补全 Imputation of missing values

<kbd><font color=Blue>SimpleImputer</font></kbd>类提供了计算缺失值的基本策略。
缺失值可以用提供的常数值计算，也可以使用缺失值所在的行/列中的统计数据(平均值、中位数或者众数)来计算。
这个类也支持不同的缺失值编码。

以下代码段演示了如何使用包含缺失值的列(轴0)的平均值来替换编码为<kbd><font color=Red>np.nan</font></kbd>的缺失值：

In [22]:
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit([[1, 2], [np.nan, 3], [7, 6]])
X = [[np.nan, 2], [6, np.nan], [7, 6]]
print(imp.transform(X))   

[[4.         2.        ]
 [6.         3.66666667]
 [7.         6.        ]]


一种更复杂的方法是使用<kbd><font color=Blue>IterativeImputer</font></kbd>类，它将每个缺失值的特征建模为其他特征的函数，并使用该估计值进行估算。它以迭代循环方式执行：在每个步骤中，将要素目标列指定为输出y，将其他列视为输入X。使用一个回归器来在已知（未缺失）ｙ的样本上，对（Ｘ，ｙ）进行拟合。然后使用这个回归器来预测缺失的ｙ值。这是以迭代的方式对每个特征进行的，然后重复<kbd><font color=Red>max_iter</font></kbd>轮。最后一轮的计算结果被返回。

In [23]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit([[1, 2], [3, 6], [4, 8], [np.nan, 3], [7, np.nan]])

X_test = [[np.nan, 2], [6, np.nan], [np.nan, 6]]
# the model learns that the second feature is double the first
print(np.round(imp.transform(X_test)))

[[ 1.  2.]
 [ 6. 12.]
 [ 3.  6.]]


<kbd><font color=Blue>KNNImputer</font></kbd>类提供了使用k近邻方法填充缺失值的插补。默认情况下，使用支持缺失值的欧几里德距离度量<kbd><font color=Red>nan_euclidean_distances</font></kbd>来查找最近的邻居。每个缺失的特征都是使用具有特征值的<kbd><font color=Red>n_neighbors</font></kbd>个近邻的值来插补的。

In [24]:
import numpy as np
from sklearn.impute import KNNImputer
nan = np.nan
X = [[1, 2, nan], [3, 4, 3], [nan, 6, 5], [8, 8, 7]]
imputer = KNNImputer(n_neighbors=2, weights="uniform")
imputer.fit_transform(X)

array([[1. , 2. , 4. ],
       [3. , 4. , 3. ],
       [5.5, 6. , 5. ],
       [8. , 8. , 7. ]])

<kbd><font color=Blue>MissingIndicator</font></kbd>转换器用于将数据集转换为相应的二进制矩阵，以指示数据集中缺失值的存在。这个变换与归算结合起来是有用的。当使用插补时，保存关于哪些值丢失的信息可以提供有用的信息。

<kbd><font color=Red>NaN</font></kbd>通常用作缺少值的占位符。但是，它强制数据类型为浮点数。参数<kbd><font color=Red>missing_values</font></kbd>允许指定其他占位符，如整数。在以下示例中，我们将使用-1作为缺失值。

In [25]:
from sklearn.impute import MissingIndicator
X = np.array([[-1, -1, 1, 3],
              [4, -1, 0, -1],
              [8, -1, 1, 0]])
indicator = MissingIndicator(missing_values=-1)
mask_missing_values_only = indicator.fit_transform(X)
mask_missing_values_only

array([[ True,  True, False],
       [False,  True,  True],
       [False,  True, False]])